In [1]:
# load the packages
import os
import yaml
import pandas as pd
import numpy as np

# load the runner 
from finetune_cgcnn import FineTune
from pymatgen.core import Structure

2024-09-27 17:06:50.083975: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-27 17:06:50.325133: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 17:06:50.325228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 17:06:50.359644: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 17:06:50.435241: I tensorflow/core/platform/cpu_feature_guar

In [2]:
zeo_df = pd.read_csv('sampled_zeolites.csv')
zeo_df

,zeo_id,largest_included_sphere,largest_free_sphere,largest_included_sphere_along_free_sphere_path,Unitcell_volume:,Density:,ASA_A^2:,NASA_A^2:,num_1d_chan,num_2d_chan,num_3d_chan,num_chan
0,h8272678,4.58324,2.95666,3.32542,3100.080,2.05975,136.9470,85.33810,0,0,0,0
1,h8306520,7.94412,3.72843,7.87277,3773.220,1.69229,597.3610,10.85110,2,0,0,2
2,h8256665,5.51967,4.64031,5.51967,5359.920,1.78699,792.5020,5.07842,4,0,0,4
3,h8276557,4.19596,3.04913,4.19596,5149.070,2.01517,575.4480,0.00000,2,0,0,2
4,h8182281,3.56851,2.89059,3.56851,2283.160,2.09756,124.0410,0.00000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
29982,h8043534,4.14026,2.93138,4.13332,767.261,2.08058,88.0899,0.00000,0,0,0,0
29983,h8092957,4.42655,2.78889,4.38478,3183.960,2.00549,205.2110,40.14450,0,0,0,0
29984,h8105206,3.61817,2.83096,3.61817,2398.600,1.99660,203.3660,30.71520,0,0,0,0
29985,h8130967,3.86190,3.03965,3.85951,2588.710,1.84997,471.4810,0.00000,2,0,0,2


In [3]:
label = 'largest_free_sphere'
zeo_df[['zeo_id', label]].to_csv(f'zeolites/{label}.csv', header = False, index = False)

In [4]:
# load config file
config = yaml.load(open("config_ft_cgcnn.yaml", "r"), Loader=yaml.FullLoader) #modify this to access cgcnn and train on cgcnn
config['batch_size'] = 64
config['dataloader']['randomSeed'] = 0

config['data_name'] = 'zeo'
config['dataset']['root_dir'] = './zeolites/cif'
config['dataset']['label_dir'] = f'./zeolites/{label}.csv'

task_name = config['data_name']
config['dataloader']['val_ratio'] = 0.10
config['dataloader']['test_ratio'] = 0.15

print(config)
# ftf: finetuning from
# ptw: pre-trained with

{'batch_size': 64, 'epochs': 100, 'eval_every_n_epochs': 1, 'fine_tune_from': 'scratch', 'log_every_n_steps': 50, 'gpu': 'cuda:0', 'random_seed': 3, 'task': 'regression', 'optim': {'optimizer': 'Adam', 'lr': 0.01, 'momentum': 0.9, 'weight_decay': '1e-6'}, 'model': {'atom_fea_len': 64, 'h_fea_len': 512, 'n_conv': 3, 'n_h': 1}, 'data_name': 'zeo', 'dataset': {'root_dir': './zeolites/cif', 'label_dir': './zeolites/largest_free_sphere.csv', 'max_num_nbr': 12, 'radius': 8, 'dmin': 0, 'step': 0.2}, 'dataloader': {'val_ratio': 0.1, 'test_ratio': 0.15, 'num_workers': 0, 'randomSeed': 0}}


In [5]:
if config['fine_tune_from'] == 'scratch':
    ftf = 'scratch'
    ptw = 'scratch'
else:
    ftf = config['fine_tune_from'].split('/')[-1]
    ptw = config['trained_with']

seed = config['dataloader']['randomSeed']

log_dir = os.path.join(
    'training_results/finetuning/CGCNN',
    'CGCNN_{}_{}_{}'.format(ptw,task_name,seed)
)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [6]:
# run the training on dataset
fine_tune = FineTune(config, log_dir)
fine_tune.train()

Running on: cuda:0
The random seed is:  3
Train size: 22490, Validation size: 2998, Test size: 4498
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [7]:
loss, metric = fine_tune.test()

training_results/finetuning/CGCNN/CGCNN_scratch_zeo_0/checkpoints/model.pth
Loaded trained model with success.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

In [8]:
# save the training results
fn = 'CGCNN_{}_{}_{}.csv'.format(ptw,task_name,seed)
print(fn)
df = pd.DataFrame([[loss, metric.item()]])
df.to_csv(
    os.path.join(log_dir, fn),
    mode='a', index=False, header=False
)

CGCNN_scratch_zeo_0.csv


In [9]:
df_test = pd.read_csv("training_results/finetuning/CGCNN/CGCNN_{}_{}_{}/test_results.csv".format(ptw, task_name, seed), header = None) #(actual, pred)
df_test.columns = ['ID', 'Actual', 'Predicted']

df_test

,ID,Actual,Predicted
0,h8272019,5.27820,4.539775
1,h8214455,6.69665,6.626317
2,h8222048,4.33286,4.241395
3,h8289329,3.93471,5.008210
4,h8092343,4.62321,3.967020
...,...,...,...
4493,h8174728,3.29892,3.675429
4494,h8219888,5.37039,4.169069
4495,h8070600,4.38707,3.601319
4496,h8289248,3.52196,4.220446


In [10]:
import scipy
from sklearn.metrics import mean_absolute_error

print(f"SRCC is {scipy.stats.spearmanr(df_test['Actual'], df_test['Predicted'])[0]}")
print(f"MAE is {mean_absolute_error(df_test['Actual'], df_test['Predicted'])}")

SRCC is 0.5400245144319634
MAE is 1.0365533097789996
